In [2]:
import cv2
import numpy as np
import mediapipe as mp

In [3]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=True,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
# file = open('log.txt', 'a')
angle_file = open('angle.txt', 'a')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
def drawTessalation(image, landmarks):
    mp_drawing.draw_landmarks(image=image,
                            landmark_list=landmarks,
                            landmark_drawing_spec=None,                          
                            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style(),
                            connections=mp_face_mesh.FACEMESH_TESSELATION)
    return image

In [6]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks, face_mesh_result


In [7]:
def display(image, title="title"):
    cv2.imshow(title, image)
    if cv2.waitKey(0) == 27:
        cv2.destroyWindow(title)

In [8]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": [landmarks[30], [landmarks[158][0], landmarks[145][1]]],
        "eye_right": [landmarks[287], [landmarks[260][0], landmarks[381][1]]],
        "shade": [landmarks[72], landmarks[124], landmarks[302], landmarks[353]]
    }
    return coordinates

In [9]:
def get_rectangle_coordinates(coordinates, angle):
    if angle == 0:
        return coordinates[0], coordinates[3]
    if angle < 0:
        return (coordinates[0][0], coordinates[2][1]), (coordinates[3][0], coordinates[1][1])
    else:
        return (coordinates[1][0], coordinates[0][1]), (coordinates[2][0], coordinates[3][1]) 

In [10]:
def add(imgCroppedMat, effectMat):
    h, w = effectMat.shape[:2]
    blend = np.zeros(effectMat.shape)
    for i in range(h):
        for j in range(w):
            if effectMat[i][j][3] == 0:
                blend[i][j] = imgCroppedMat[i][j]
            else:
                blend[i][j] = effectMat[i][j]
    return blend

In [11]:
def add_spec_effect(image, specMat, top_left, bottom_right):
    cropped_image = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0], :]
    w, h, _ = cropped_image.shape
    specs = cv2.resize(specMat, (h, w))
    blend = cv2.addWeighted(cropped_image, 0, specs, 1.0, 0)
    # blend = add(cropped_image, specs)
    return blend

In [12]:
def remove_whitespace(image, blend, x, y, threshold=225):
    for i in range(blend.shape[0]):
        for j in range(blend.shape[1]):
            for k in range(3):
                if blend[i][j][k] > threshold:
                    blend[i][j][k] = image[i + y][j + x][k]

In [13]:
import math
def get_angle(coordinates):
    height = (coordinates[1][1] - coordinates[2][1])
    base = (coordinates[1][0] - coordinates[2][0])
    
    angle = math.atan(height/base) * 180/math.pi
    return angle
    

In [14]:
def get_rotated_image(im, angle):
    mask = (im == [0,0,0,0]).all(axis=2)
    im[mask] = [255, 255, 255, 0]
    
    imHeight, imWidth = im.shape[0], im.shape[1]
    centreX, centreY = imWidth//2, imHeight//2
        
    rotationMat = cv2.getRotationMatrix2D(
        center=(centreX, centreY),
        angle=angle,
        scale=1
    )
    
    cos = np.abs(rotationMat[0][0])
    sin = np.abs(rotationMat[1][0])
    
    newWidth = int((imHeight * sin) + (imWidth * cos))
    newHeight = int((imHeight * cos) + (imWidth * sin))
    
    rotationMat[0][2] += newWidth/2 - centreX
    rotationMat[1][2] += newHeight/2 - centreY
    
    dst_mat = np.full((newHeight, newWidth, 4), [255, 255, 255, 0])
    rotatedMat = cv2.warpAffine(
        im,
        rotationMat,
        (newWidth, newHeight),
        np.ascontiguousarray(dst_mat, np.uint8),
        borderMode=cv2.BORDER_TRANSPARENT
    )
    # cv2.imshow("title", rotatedMat)
    # if cv2.waitKey(0) == 27:
    #     cv2.destroyWindow("title")
    return rotatedMat[:, ::-1], (newHeight, newWidth)

In [15]:

def draw_specs_effect(image, coordinates, angle):
    rotated_image, shape = get_rotated_image(
        cv2.imread(
            'effects/example1.png',
            cv2.IMREAD_UNCHANGED
        ),
        angle
    )
    pt1, pt2 = get_rectangle_coordinates(coordinates, angle)
    
    blend = add_spec_effect(image, rotated_image, pt1, pt2)
    remove_whitespace(
        image=image,
        blend=blend,
        x=pt1[0],
        y=pt1[1], 
        threshold=150
    )
    image[pt1[1]: pt2[1], pt1[0]:pt2[0], :] = blend
    return (angle, pt1, pt2)
    

In [16]:
def draw_coordinates(image, points, color = [150, 0, 200]):
    neighbor_vector = [(0,0), (1,0), (0,1), (-1,0), (0, -1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for pt in points:
        for i in neighbor_vector:
            image[pt[1] + i[1] ][pt[0] + i[0]] = color
    return image

In [17]:
# cam = cv2.VideoCapture(0)

# i = 0
# while True:
#     ret, frame = cam.read()
    
#     if ret:
#         height, width, _ = frame.shape
#         image = cv2.resize(frame, (width//2, height//2))[:,::-1]
#         landmarks = get_landmarks(image=image)
#         coordinates = get_coordinates(landmarks=landmarks)
#         print(coordinates['spec'])
#         # draw_iris_effects(image, coordinates)
#         # draw_specs_effect(image, coordinates)
#         image = draw_coordinates(image, coordinates['spec'], color=[255, 255, 255])
        
#         cv2.imshow("Live face effects", image)
#         if cv2.waitKey(5) & 0xFF == 27 or i == 1:
#             cv2.destroyAllWindows() 
#             cam.release()
#             break

In [23]:
import sys


cam = cv2.VideoCapture(0)
i = 0
s = []
neutral_angle = 0
while cam.isOpened():
    s.clear()
    s.append("{} -> ".format(i))
    ret, frame = cam.read()
    if ret:
        # frame = frame[:, ::-1]
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)[:, ::-1]
        
        landmarks, face_mesh_result = get_landmarks(image=frame)
        try:
            coordinates = get_coordinates(landmarks=landmarks)
            
            # _img = draw_coordinates(frame, [landmarks[72], landmarks[117], landmarks[302], landmarks[346]], [255, 255, 255, 255])

            angle = get_angle(coordinates['shade'])
            
            if neutral_angle == 0:
                neutral_angle += angle
                print(neutral_angle)
                # angle_file.write("{} -> {}\n".format(i, neutral_angle))
            
            # rect_coords = get_rectangle_coordinates(
            #     [landmarks[72], landmarks[117], landmarks[302], landmarks[346]],
            #     angle - neutral_angle
            # )
            _img = draw_coordinates(
                frame.copy(), 
                coordinates['shade'],
                [255, 255, 255, 255]
            )
            
            # print(coordinates['shade'], ((rect_coords[1])))
            
            # _img = draw_coordinates(frame.copy(), rect_coords, [255, 255, 255, 255])
            # _img = cv2.putText(
            #     _img.astype(np.uint8),
            #     str(angle - neutral_angle),
            #     (20, 20),
            #     cv2.FONT_HERSHEY_SIMPLEX, 
            #     1,
            #     (255, 255, 255),
            #     1
            # )
            # img = drawTessalation(
            #     np.ascontiguousarray(frame.copy()[:, :, :3], np.uint8),
            #     face_mesh_result.multi_face_landmarks[0]
            # )
            
            # print(type(img), img.shape, type(frame), frame.shape)
            
            result = draw_specs_effect(frame, coordinates['shade'], angle - neutral_angle)
            # im_show = np.concatenate(img, frame[:, :, :3])
            cv2.imshow('Landmarks', _img)
        except:
            print("No eyes landmarks detected, {}".format(sys.exc_info()))
            
        # s.append('original coordinates => {}\n'.format(coordinates['shade']))
        # s.append('effect coordinates => {}\n'.format(coordinates['spec']))
        # # s.append('angle => {}\n'.format(angle))
        # s.append('---> {}\n'.format(result))
        # # s.append('replace cooridnates => {} - {}\n'.format(pt1, pt2))
        # s.append('\n----------------------------------------------------------------\n\n')
        
        # file.writelines(s)
        # i+=1
        cv2.imshow('Live face effects', frame)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows()
            cam.release()
            break

-23.085133672808276


QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to target thread (0x5561bddea570)

QObject::moveToThread: Current thread (0x5561bddea570) is not the object's thread (0x5561be0a4a30).
Cannot move to tar

In [19]:
cam.release()

In [20]:
# image = cv2.imread('images/img.jpg')

# landmarks = get_landmarks(image)


# # for i in range(1, 469):
# _img = draw_coordinates(image, [landmarks[72], landmarks[117], landmarks[302], landmarks[346]], [255, 255, 255])

# cv2.imshow('Live face effects', _img)
# if cv2.waitKey(0) == 27:
#     cv2.destroyAllWindows()

In [21]:
import time

In [22]:

if neutral_angle == "":
    neutral_angle = 0

frame = cv2.imread('images/img.jpg')[:, ::-1]
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)[:, ::-1]
        
landmarks = get_landmarks(image=frame)
coordinates = get_coordinates(landmarks=landmarks)

_img = draw_coordinates(frame, [landmarks[72], landmarks[117], landmarks[302], landmarks[346]], [255, 255, 255])

angle = get_angle([landmarks[72], landmarks[117], landmarks[302], landmarks[346]])
if neutral_angle == "":
    print(angle)
    neutral_angle = angle
    angle_file.write('{} -> {}\n'.format(time.time(), int(angle)))
rect_coords = get_rectangle_coordinates(
    [landmarks[72], landmarks[117], landmarks[302], landmarks[346]],
    angle + angle
)
_img = draw_coordinates(
    _img,
    rect_coords
)

print(coordinates['shade'], ((rect_coords[1])))

# _img = draw_coordinates(frame, rect_coords, [255, 255, 255, 255])
_img = cv2.putText(
    _img.astype(np.uint8),
    str(angle + neutral_angle),
    (20, 20),
    cv2.FONT_HERSHEY_SIMPLEX, 
    1,
    (255, 255, 255),
    1
)

cv2.imshow("image", _img)
if cv2.waitKey(0) == 27:
    cv2.destroyAllWindows()

IndexError: tuple index out of range

In [ ]:
angle_file.close()